An dieser Stelle werden alle notwendigen Imports getätigt.

In [ ]:
import chess
import chess.engine
import random
import chess.svg
import time
import sys
from IPython.display import display, HTML, clear_output

Globale Konstanten

In [ ]:
TIMEOUT_SECONDS: int = 6

Globale Variablen

In [ ]:
best_move = None
current_depth = 0
global_best_move = None
is_timeout: bool = False
start_time: float = 0.0

Nachfolgend werden die Figurenwerte definiert.

In [ ]:
piece_values = {
    chess.BISHOP: 330,
    chess.KING: 20_000,
    chess.KNIGHT: 320,
    chess.PAWN: 100,
    chess.QUEEN: 900,
    chess.ROOK: 500,
}

Die Funktion `get_piece_value` gibt den Figurenwert für eine übergebene Figur auf dem Schachbrett zurück.

In [ ]:
def get_piece_value(piece: chess.Piece) -> int:
    if not piece:
        return 0
    factor = -1 if piece.color == chess.BLACK else 1
    return factor * piece_values.get(piece.piece_type)

Nachfolgend werden die Piece-Squared Tables definiert.

In [ ]:
piece_squared_tables = {
    chess.BISHOP: (
        (-20, -10, -10, -10, -10, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,  10,  10,   5,   0, -10),
        (-10,   5,   5,  10,  10,   5,   5, -10),
        (-10,   0,  10,  10,  10,  10,   0, -10),
        (-10,  10,  10,  10,  10,  10,  10, -10),
        (-10,   5,   0,   0,   0,   0,   5, -10),
        (-20, -10, -10, -10, -10, -10, -10, -20),
    ),
    chess.KING: (
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-20, -30, -30, -40, -40, -30, -30, -20),
        (-10, -20, -20, -20, -20, -20, -20, -10),
        ( 20,  20,   0,   0,   0,   0,  20,  20),
        ( 20,  30,  10,   0,   0,  10,  30,  20),
    ),
    chess.KNIGHT: (
        (-50, -40, -30, -30, -30, -30, -40, -50),
        (-40, -20,   0,   0,   0,   0, -20, -40),
        (-30,   0,  10,  15,  15,  10,   0, -30),
        (-30,   5,  15,  20,  20,  15,   5, -30),
        (-30,   0,  15,  20,  20,  15,   0, -30),
        (-30,   5,  10,  15,  15,  10,   5, -30),
        (-40, -20,   0,   5,   5,   0, -20, -40),
        (-50, -40, -30, -30, -30, -30, -40, -50),
    ),
    chess.PAWN: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        ( 50,  50,  50,  50,  50,  50,  50,  50),
        ( 10,  10,  20,  30,  30,  20,  10,  10),
        (  5,   5,  10,  25,  25,  10,   5,   5),
        (  0,   0,   0,  20,  20,   0,   0,   0),
        (  5,  -5, -10,   0,   0, -10,  -5,   5),
        (  5,  10,  10, -20, -20,  10,  10,   5),
        (  0,   0,   0,   0,   0,   0,   0,   0),
    ),
    chess.QUEEN: (
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,   5,   5,   5,   0, -10),
        ( -5,   0,   5,   5,   5,   5,   0,  -5),
        (  0,   0,   5,   5,   5,   5,   0,  -5),
        (-10,   5,   5,   5,   5,   5,   0, -10),
        (-10,   0,   5,   0,   0,   0,   0, -10),
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
    ),
    chess.ROOK: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        (  5,  10,  10,  10,  10,  10,  10,   5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        (  0,   0,   0,   5,   5,   0,   0,   0),
    ),
}

In der Endphase wird für den König eine andere Tabelle verwendet. Diese ist nachfolgend definiert.

In [ ]:
kings_end_game_squared_table = (
    (-50, -40, -30, -20, -20, -30, -40, -50),
    (-30, -20, -10,   0,   0, -10, -20, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -30,   0,   0,   0,   0, -30, -30),
    (-50, -30, -30, -30, -30, -30, -30, -50),
)

Die Funktion `get_piece_quared_tables_value` liefert den Wert der Piece-Squared Table für eine übergebene Figur auf dem Schachbrett.

In [ ]:
def get_piece_squared_tables_value(piece: chess.Piece, end_game: bool=False) -> int:
    if not piece:
        return 0
    factor = -1 if piece.color == chess.BLACK else 1
    row = piece.piece_type // 8
    column = piece.piece_type % 8
    
    if end_game and piece.piece_type == chess.KING:
        return kings_end_game_squared_table[row][column]
    
    piece_squared_table = piece_squared_tables.get(piece.piece_type)
    return factor * piece_squared_table[row][column]

Die Funktion `simple_eval_heuristic` setzt die einfache Bewertungsheuristik um.

In [ ]:
def simple_eval_heuristic(board: chess.Board, end_game: bool=False) -> int:
    piece_value = 0
    for square in range(64):
        piece = board.piece_at(square)
        piece_value += get_piece_value(piece)
        piece_value += get_piece_squared_tables_value(piece, end_game)
    return piece_value

Implementierung des Iterative Deepening

In [ ]:
def iterative_deepening(board: chess.Board, depth: int):
    global best_move
    global current_depth
    global global_best_move
    global is_timeout
    global start_time
    
    is_timeout = False
    start_time = time.time()
    d = 0

    while True:
        d += 1
        if d > 1:
            global_best_move = best_move
            print(f"Completed search with depth {current_depth}. Best move so far: {global_best_move}")
        current_depth = depth + d
        minimize(board, current_depth, -sys.maxsize, sys.maxsize)

        if is_timeout:
            return global_best_move

Implementierung der `maximize` Funktion.

In [ ]:
def maximize(board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    global is_timeout
    global start_time
    
    if time.time() - start_time > TIMEOUT_SECONDS:
        is_timeout = True
        return alpha

    if depth < 1:
        return simple_eval_heuristic(board)
    
    score = alpha
    moves = board.legal_moves

    for move in moves:
        newboard = board.copy()
        newboard.push(move)
        move_score = minimize(newboard, depth - 1, alpha, beta)
        newboard.pop()

        if move_score > score:
            score = move_score
            
            if score > alpha:
                alpha = score

        if score >= beta:
            break
    return score

Implementierung der `minimize` Funktion.

In [ ]:
def minimize(board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    global best_move
    global global_best_move
    
    if depth < 1:
        return simple_eval_heuristic(board)
    
    score = beta
    moves = board.legal_moves

    for move in moves:
        newboard = board.copy()
        newboard.push(move)
        move_score = maximize(newboard, depth - 1, alpha, beta)
        newboard.pop()

        if move_score < score:
            score = move_score
            best_move = move
            
            if score < beta:
                beta = score
                global_best_move = best_move

        if score <= alpha:
            break
    return score

`reset` ist eine Hilfsfunktion, die die globalen Werte für alpha und beta zurücksetzt.

In [ ]:
def reset():
    global positions_analyzed_alpha_beta
    global positions_analyzed_minimax
    positions_analyzed_alpha_beta = 0
    positions_analyzed_minimax = 0

`who` ist eine Hilfsfunktion, um den Namen des Spielers (Weiß oder Schwarz) zurückzuliefern. Dies wird für die Anzeige benötigt.

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

`get_move` ist eine Hilfsfunktion, die die Nutzereingabe in einen Zug umwandelt, wenn dies möglich ist.

In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

Die Funktion `human_player` repräsentiert den menschlichen Spieler und koordiniert die Züge des Spielers.

In [ ]:
def human_player(board):
    display(board)
    uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print(f"Legal moves: {(', '.join(sorted(legal_uci_moves)))}")
        uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    return uci

Die Funktion `ai_player` repräsentiert den Schachcomputer und koordiniert den Ablauf seines Entscheidungsprozesses.

In [ ]:
def ai_player(board):
    return iterative_deepening(board, 0)

`play_game` ist die Funktion, die den Spielablauf koordiniert.

In [ ]:
def play_game(pause=0.3):
    board = chess.Board()
    
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                move = board.parse_uci(human_player(board))
            else:
                move = ai_player(board)
                
            name = who(board.turn)
            board.push(move)
            html = f"{board}"
            
            reset()
            clear_output(wait=True)
            display(HTML(html))
            time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted"
        return None, msg, board
    
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: fivefold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    
    print(msg)
    return result, msg, board

In [ ]:
play_game()